In [1]:
suppressPackageStartupMessages(library('tidyverse'))
# suppressPackageStartupMessages(library('tidylog'))

# Mostra valores sem notação científica
options(scipen=999)

library('sf')
library('raster')

options(repr.matrix.max.cols=150) # repr.matrix.max.rows=100

Linking to GEOS 3.9.1, GDAL 3.2.2, PROJ 8.0.0

Loading required package: sp


Attaching package: ‘raster’


The following object is masked from ‘package:dplyr’:

    select


The following object is masked from ‘package:tidyr’:

    extract




In [2]:
# # Gravar todos os shapes dos municípios em gpkg
# open_path <- '../../../indice-mobilidade_dados/01_municipios/2019'
# for (f in list.files(open_path)){
#     open_file <- sprintf('%s/%s', open_path, f)
#     out_file <- sprintf('%s.gpkg', open_file)
#     municipios <- read_rds(open_file)
#     st_write(municipios, out_file, driver = 'GPKG', append = FALSE)
# }

# 05_censo_escolar

In [3]:
open_file <- '../../../indice-mobilidade_dados/05_censo_escolar/2019/educacao_2019_filter_geocoded_gmaps.rds'
censo_escolar <- read_rds(open_file)

# Criar id único para latlong
censo_escolar <- 
    censo_escolar %>% 
    mutate(latitude = str_sub(as.character(latitude), start = 0, end = 10),
           longitude = str_sub(as.character(longitude), start = 0, end = 10),
           latlong_id = sprintf('%s%s', latitude, longitude)) %>% 
    add_column(index_col = 1:nrow(censo_escolar), .before = 'co_entidade')

dim(censo_escolar)
head(censo_escolar, 2)

[1] 11236    14

,index_col,co_entidade,ano,code_muni,no_entidade,mat_infantil,mat_fundamental,mat_medio,endereco,uf,municipio,latitude,longitude,latlong_id
,<int>,<dbl>,<dbl>,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,13025120,2019,1302603,CMEI HUMBERTO DE ALENCAR CASTELO BRANCO,339,0,0,"R. 22, S/N CONJUNTO CASTELO BRA. PARQUE 10 DE NOVEMBRO. 69055-400 Manaus - AM.",AM,Manaus,-3.0820916,-60.009773,-3.0820916-60.009773
2,2,13025180,2019,1302603,EM FRANCISCA GOMES MENDES,0,430,0,"R. BELO HORIZONTE, S/N COMPENSA. 69035-100 Manaus - AM.",AM,Manaus,-3.0975711,-60.061629,-3.0975711-60.061629


In [4]:
censo_escolar %>% 
    filter(municipio == 'Teresina') %>% 
    mutate(n_vagas = mat_infantil + mat_fundamental + mat_medio) %>% 
    summarize(vagas = sum(n_vagas))

vagas
<int>
129050


In [5]:
equiv_muni <- 
    censo_escolar %>% 
    mutate(IBGE = str_sub(code_muni, start = 0, end = 6)) %>% 
    dplyr::select(IBGE, municipio) %>%     
    distinct() %>% 
    filter(!is.na(municipio)) %>% 
    arrange(municipio)

dim(equiv_muni)
equiv_muni

[1] 21  2

IBGE,municipio
<chr>,<chr>
310620,Belo Horizonte
350950,Campinas
500270,Campo Grande
410690,Curitiba
230440,Fortaleza
520870,Goiânia
250750,João Pessoa
411370,Londrina
270430,Maceió


In [6]:
lats_repetidos_educ <- 
    censo_escolar %>% 
    group_by(latlong_id) %>% 
    tally() %>% 
    filter(n > 1)

dim(lats_repetidos_educ)
head(lats_repetidos_educ, 1)

[1] 30  2

latlong_id,n
<chr>,<int>
-12.90436-38.40622,2


In [7]:
censo_escolar %>% filter(.$latlong_id %in% lats_repetidos_educ$latlong_id) %>% group_by(municipio) %>% tally()

municipio,n
<chr>,<int>
Rio de Janeiro,6
Salvador,2
São Paulo,53
Teresina,4


In [8]:
censo_escolar %>% 
    filter(.$latlong_id %in% lats_repetidos_educ$latlong_id & municipio == 'Salvador') %>% 
    dplyr::select(-c(ano, code_muni, uf, municipio, latlong_id)) %>% 
    arrange(endereco)

index_col,co_entidade,no_entidade,mat_infantil,mat_fundamental,mat_medio,endereco,latitude,longitude
<int>,<dbl>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>
3344,29336708,EE - COLEGIO ESTADUAL EDUARDO BAHIANA,0,817,534,"NULL ESTRADA DO COQUEIRO GRANDE - JAGUARIPE, FAZENDA GRANDE II. 41340-000 Salvador - BA.",-12.90436,-38.40622
2933,29182115,EE - COLEGIO DA POLICIA MILITAR- CPM DONA LEONOR CALMON,0,352,539,"R. CONJ. FAZENA GRANDE II - JAGUARIPE, S/N PD,PD,PD. FAZENDA GRANDE II. 41310-355 Salvador - BA.",-12.90436,-38.40622


In [9]:
censo_escolar %>% 
    filter(.$latlong_id %in% lats_repetidos_educ$latlong_id & municipio == 'Teresina') %>% 
    dplyr::select(-c(ano, code_muni, uf, municipio)) %>% 
    arrange(latlong_id)

index_col,co_entidade,no_entidade,mat_infantil,mat_fundamental,mat_medio,endereco,latitude,longitude,latlong_id
<int>,<dbl>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
824,22025057,ESCOLA MUNICIPAL SANTA FILOMENA,0,268,0,"POVOADO VOADO DOIS IRMAOS, PI 112 KM 18. POVOADO DOIS IRMAOS. 64000-180 Teresina - PI.",-4.917726,-42.783535,-4.917726-42.783535
1056,22135553,U E SANTA FILOMENA,0,0,183,"LOC DOIS IRMAO EIXO SOCOPO, PI112 KM18 SUP NORDESTE. SOCOPO. 64099-899 Teresina - PI.",-4.917726,-42.783535,-4.917726-42.783535
1005,22124071,CENTRO MUNICIPAL DE EDUCACAO INFANTIL TAPUIA,54,0,0,"POVOADO VOADO TAPUIA, POVOADO TAPUIA. 64000-180 Teresina - PI.",-5.08768,-42.797958,-5.08768-42.797958
1147,22180605,CENTRO MUNICIPAL DE EDUCACAO INFANTIL ANTONIO ALVES DE HOLANDA,155,0,0,"NOVA CAJAIBA, S/N ESTRADA DA CACIMBA V. 64063-015 Teresina - PI.",-5.08768,-42.797958,-5.08768-42.797958


In [10]:
censo_escolar %>% 
    filter(.$latlong_id %in% lats_repetidos_educ$latlong_id & municipio == 'Rio de Janeiro') %>% 
    dplyr::select(-c(ano, code_muni, uf, municipio)) %>% 
    arrange(latlong_id)

index_col,co_entidade,no_entidade,mat_infantil,mat_fundamental,mat_medio,endereco,latitude,longitude,latlong_id
<int>,<dbl>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
4947,33076502,ESCOLA TECNICA ESTADUAL VISCONDE DE MAUA,0,0,641,"RUA JOAO VICENTE, 1775 MARECHAL HERMES. 21610-210 Rio de Janeiro - RJ.",-22.85925,-43.37525,-22.85925-43.37525
5751,33126203,E E DE ENS FUND VISCONDE DE MAUA,0,451,0,"RUA XAVIER CURADO, MARECHAL HERMES. 21610-330 Rio de Janeiro - RJ.",-22.85925,-43.37525,-22.85925-43.37525
5973,33163570,0625804 EDI PROFESSORA HELIA LUCIA MORENO FERREIRA,215,0,0,"GIL EANES, S/N PAVUNA. 21532-210 Rio de Janeiro - RJ.",-22.906846,-43.172896,-22.906846-43.172896
6142,33179212,0918831 EDI ELFLODIZIO DOS SANTOS,328,0,0,"RUA DO GOIABAL, 75 NOVA CIDADE. INHOAIBA. 23063-650 Rio de Janeiro - RJ.",-22.906846,-43.172896,-22.906846-43.172896
6145,33179247,0918834 EDI ENGENHEIRA ELAINE MONSORES PIRES,299,0,0,"RUA ITAUNAS, CAMPO GRANDE. 23068-155 Rio de Janeiro - RJ.",-22.906846,-43.172896,-22.906846-43.172896
6150,33179360,1026810 EDI ALCEBIADES FRANCISCO ROSA,504,0,0,"RUA ANTONIO RAIOL, 356 GUARATIBA. 23032-090 Rio de Janeiro - RJ.",-22.906846,-43.172896,-22.906846-43.172896
4721,33071829,CPII - CAMPUS ENGENHO NOVO II,0,542,523,"RUA BARAO DO BOM RETIRO, 726 ENGENHO NOVO. 20715-003 Rio de Janeiro - RJ.",-22.90989,-43.26988,-22.90989-43.26988
4722,33071837,CPII - CAMPUS ENGENHO NOVO I,0,499,0,"RUA BARAO DO BOM RETIRO, 726 ENGENHO NOVO. 20715-003 Rio de Janeiro - RJ.",-22.90989,-43.26988,-22.90989-43.26988


In [11]:
censo_escolar %>% 
    filter(.$latlong_id %in% lats_repetidos_educ$latlong_id & municipio == 'São Paulo') %>% 
    dplyr::select(-c(ano, code_muni, uf, municipio, latitude, longitude)) %>% 
    arrange(latlong_id)

index_col,co_entidade,no_entidade,mat_infantil,mat_fundamental,mat_medio,endereco,latlong_id
<int>,<dbl>,<chr>,<int>,<int>,<int>,<chr>,<chr>
7724,35058075,CEU EMEF PERUS,0,811,0,"BERNARDO JOSE DE LORENA, S/N VILA FANTON. 5203200 São Paulo - SP.",-23.4061-46.7517
7725,35058087,CEU CEI PERUS,233,0,0,"BERNARDO JOSE DE LORENA, S/N VILA FANTON. 5203200 São Paulo - SP.",-23.4061-46.7517
8797,35242226,CEU EMEI JORGE AMADO,484,0,0,"BERNARDO JOSE DE LORENA, S/N VILA FANTON. 5203200 São Paulo - SP.",-23.4061-46.7517
7721,35058002,CEU EMEF PERA MARMELO,0,768,0,"PERAMARMELO, 226 4 ANDAR. JARDIM SANTA LUCRECIA. 5185420 São Paulo - SP.",-23.4364-46.7503
7723,35058063,CEU CEI PERA MARMELO,318,0,0,"PERAMARMELO, 226 4 ANDAR. JARDIM SANTA LUCRECIA. 5185420 São Paulo - SP.",-23.4364-46.7503
9115,35397088,CEU CEI PARQUE ANHANGUERA,222,0,0,"PEDRO JOSE DE LIMA, 1020 JARDIM ANHANGUERA. 5267174 São Paulo - SP.",-23.44-46.7873
9117,35397271,CEU EMEF PARQUE ANHANGUERA,0,957,0,"PEDRO JOSE DE LIMA, 1020 JARDIM ANHANGUERA. 5267174 São Paulo - SP.",-23.44-46.7873
9040,35298268,CEU EMEF JACANA,0,1069,0,"FRANCISCA ESPOSITO TONETTI, S/N GUAPIRA. 2276010 São Paulo - SP.",-23.457175-46.583861
9042,35298281,CEU CEI JACANA,295,0,0,"FRANCISCA ESPOSITO TONETTI, S/N GUAPIRA. 2276010 São Paulo - SP.",-23.457175-46.583861


# 06_cnes_saude

In [8]:
open_file <- '../../../indice-mobilidade_dados/06_cnes_saude/2019/saude_2019_filter_geocoded.rds'
cnes_saude <- read_rds(open_file)

# Criar id único para latlong
cnes_saude <- 
    cnes_saude %>% 
    mutate(latitude = str_sub(as.character(NU_LATITUDE), start = 1, end = 10),
           longitude = str_sub(as.character(NU_LONGITUDE), start = 1, end = 10),
           latlong_id = sprintf('%s%s', latitude, longitude))

dim(cnes_saude)
head(cnes_saude, 2)

[1] 6761   86

CO_UNIDADE,IBGE,CO_CNES,NU_CNPJ_MANTENEDORA,TP_PFPJ,NIVEL_DEP,NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_COMPLEMENTO,NO_BAIRRO,CO_CEP,CO_REGIAO_SAUDE,CO_MICRO_REGIAO,CO_DISTRITO_SANITARIO,CO_DISTRITO_ADMINISTRATIVO,NU_TELEFONE,NU_FAX,NO_EMAIL,NU_CPF,NU_CNPJ,CO_ATIVIDADE,CO_CLIENTELA,NU_ALVARA,DT_EXPEDICAO,TP_ORGAO_EXPEDIDOR,DT_VAL_LIC_SANI,TP_LIC_SANI,TP_UNIDADE,CO_TURNO_ATENDIMENTO,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')",CO_USUARIO,CO_CPFDIRETORCLN,REG_DIRETORCLN,ST_ADESAO_FILANTROP,CO_MOTIVO_DESAB,NO_URL,NU_LATITUDE,NU_LONGITUDE,"TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')",NO_USUARIO_GEO,CO_NATUREZA_JUR,TP_ESTAB_SEMPRE_ABERTO,ST_GERACREDITO_GERENTE_SGIF,ST_CONEXAO_INTERNET,NO_FANTASIA_ABREV,TP_GESTAO,"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",CO_TIPO_ESTABELECIMENTO,CO_ATIVIDADE_PRINCIPAL,ST_CONTRATO_FORMALIZADO,CO_TIPO_UNIDADE,DS_TIPO_UNIDADE,CO_GESTAO_00,CO_GESTAO_01,CO_GESTAO_02,CO_GESTAO_03,CO_GESTAO_04,CO_GESTAO_05,CO_GESTAO_06,TP_PROG_0,TP_PROG_1,TP_PROG_2,CO_CONVENIO_01,CO_CONVENIO_02,CO_CONVENIO_03,CO_CONVENIO_04,CO_CONVENIO_05,CO_CONVENIO_06,CO_CONVENIO_07,CO_ATENDIMENTO_PRESTADO_01,CO_ATENDIMENTO_PRESTADO_02,CO_ATENDIMENTO_PRESTADO_03,CO_ATENDIMENTO_PRESTADO_04,CO_ATENDIMENTO_PRESTADO_05,CO_ATENDIMENTO_PRESTADO_06,CO_ATENDIMENTO_PRESTADO_07,health_low,health_med,health_high,latitude,longitude,latlong_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
2611607810032,261160,7810032,NA,3,1,FARIAS ANALISES CLINICAS EIRELI,FARIAS ANALISES CLINICAS,AVENIDA NORTE MIGUEL ARRAES DE ALENCAR,2535,NA,ENCRUZILHADA,52041080,1,1,02,NA,NA,NA,erikacfarias@laboaratorio.ciac.com.br,NA,22235187000145,04,03,0705340117,18-fev-2019 00:00:00,2,18-fev-2020 00:00:00,1,36,03,26,261160,30/12/2019,FLAVIA,07663846400,0140,NA,NA,NA,-8.041001,-34.88614,30/12/2019,FLAVIA,2305,N,NA,S,NA,M,04/09/2015,018,002,S,36,CLINICA/CENTRO DE ESPECIALIDADE,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,-8.0410008,-34.886136,-8.0410008-34.886136
1302607695667,130260,7695667,NA,3,1,I S DE SOUZA,AMAZON CLINICAL CENTER,R BERNARDO MICHILES,1296,SALA 5 2 ANDAR,PETROPOLIS,69067000,001,NA,NA,NA,92 33467384,NA,amazonclinical@gmail.com,NA,13696457000124,04,NA,NA,NA,NA,NA,NA,60,NA,13,130260,27/10/2017,AMAZON,43942040204,AM-011527-0,NA,04,NA,NA,NA,NA,NA,2062,S,NA,S,NA,E,13/04/2015,000,022,NA,NA,NA,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,NA,NA,NANA


In [9]:
lats_repetidos_saude <- 
    cnes_saude %>% 
    group_by(latlong_id) %>% 
    tally() %>% 
    filter(n > 1) %>% 
    arrange(-n)

dim(lats_repetidos_saude)
head(lats_repetidos_saude, 20)

[1] 246   2

latlong_id,n
<chr>,<int>
NANA,756
-23.548-46.636,471
-22.903-43.208,125
-8.054-34.881,94
-5.795-35.209,76
-19.817-43.956,68
-23.179-45.887,64
-7.115-34.863,61
-5.089-42.802,59


In [10]:
cnes_saude %>% 
    filter(.$latlong_id %in% lats_repetidos_saude$latlong_id & latlong_id != 'NANA') %>% 
    group_by(IBGE) %>% 
    tally() %>% 
    left_join(equiv_muni, by = 'IBGE') %>% 
    arrange(IBGE)

IBGE,n,municipio
<chr>,<int>,<chr>
130260,113,Manaus
221100,106,Teresina
230440,6,Fortaleza
240810,93,Natal
250750,65,João Pessoa
261160,131,Recife
270430,86,Maceió
292740,104,Salvador
310620,138,Belo Horizonte


In [15]:
muni = 'Manaus'
cnes_saude %>% 
    filter(.$latlong_id %in% lats_repetidos_saude$latlong_id & latlong_id != 'NANA') %>% 
    filter(IBGE == '130260') %>% 
    dplyr::select(NO_FANTASIA, NO_LOGRADOURO, NU_ENDERECO, NO_BAIRRO, CO_CEP, latlong_id) %>% 
    mutate(numero = str_replace(NU_ENDERECO, 'S/N', ''),
           end_geocode = sprintf('%s, %s, %s, %s, %s', NO_LOGRADOURO, numero, NO_BAIRRO, muni, CO_CEP)) %>% 
    arrange(latlong_id) %>% 
    head(40)

NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_BAIRRO,CO_CEP,latlong_id,numero,end_geocode
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
UNIDADE DE SUPORTE BASICO USB 32,RUA NOSSA SENHORA FATIMA,1995,SANTA ETELVINA,69059420,-2.9849401-60.001826,1995,"RUA NOSSA SENHORA FATIMA, 1995, SANTA ETELVINA, Manaus, 69059420"
UNIDADE DE SUPORTE AVANCADO USA 06,RUA NOSSA SENHORA FATIMA,1995,SANTA ETELVINA,69059420,-2.9849401-60.001826,1995,"RUA NOSSA SENHORA FATIMA, 1995, SANTA ETELVINA, Manaus, 69059420"
UNIDADE DE SUPORTE BASICO USB 10,RUA NOSSA SENHORA FATIMA,1995,SANTA ETELVINA,69059420,-2.9849401-60.001826,1995,"RUA NOSSA SENHORA FATIMA, 1995, SANTA ETELVINA, Manaus, 69059420"
UNIDADE DE SUPORTE BASICO USB 06,RUA BEIJA FLOR,S/N,CIDADE DE DEUS,69099000,-3.0084199-59.939539,,"RUA BEIJA FLOR, , CIDADE DE DEUS, Manaus, 69099000"
UNIDADE DE SUPORTE BASICO USB 15,RUA BEIJA FLOR,S/N,CIDADE DE DEUS,69099000,-3.0084199-59.939539,,"RUA BEIJA FLOR, , CIDADE DE DEUS, Manaus, 69099000"
UNIDADE DE SUPORTE BASICO USB 08,RUA 143,S/N,CIDADE NOVA,69099210,-3.0323795-59.964622,,"RUA 143, , CIDADE NOVA, Manaus, 69099210"
UNIDADE DE SUPORTE BASICO USB 24,RUA 143,S/N,CIDADE NOVA,69099210,-3.0323795-59.964622,,"RUA 143, , CIDADE NOVA, Manaus, 69099210"
UNIDADE DE SUPORTE BASICO USB 33,RUA 143,33,CIDADE NOVA,69099210,-3.0323795-59.964622,33,"RUA 143, 33, CIDADE NOVA, Manaus, 69099210"
UNIDADE DE SUPORTE AVANCADO USA 05,RUA 143,S/N,CIDADE NOVA,69099210,-3.0323795-59.964622,,"RUA 143, , CIDADE NOVA, Manaus, 69099210"


In [16]:
cnes_saude %>% 
    filter(.$latlong_id %in% lats_repetidos_saude$latlong_id & latlong_id != 'NANA') %>% 
    group_by(latlong_id, IBGE) %>% 
    tally() %>% 
    left_join(equiv_muni, by = 'IBGE') %>% 
    filter(n > 3) %>% 
    arrange(-n)

latlong_id,IBGE,n,municipio
<chr>,<chr>,<int>,<chr>
-23.548-46.636,355030,471,São Paulo
-22.903-43.208,330455,125,Rio de Janeiro
-8.054-34.881,261160,94,Recife
-5.795-35.209,240810,76,Natal
-19.817-43.956,310620,68,Belo Horizonte
-23.179-45.887,354990,64,São José dos Campos
-7.115-34.863,250750,61,João Pessoa
-5.089-42.802,221100,59,Teresina
-3.102-60.025,130260,46,Manaus


In [17]:
cnes_saude %>% 
    filter(.$latlong_id %in% lats_repetidos_saude$latlong_id & IBGE == '230440' & latlong_id != 'NANA') %>% 
    dplyr::select(c(NO_RAZAO_SOCIAL, NO_FANTASIA, NO_LOGRADOURO, NU_ENDERECO, NO_BAIRRO, CO_CEP, latlong_id)) %>% 
    arrange(latlong_id)

NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_BAIRRO,CO_CEP,latlong_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
SECRETARIA DA SAUDE DO ESTADO DO CEARA,COORD REGIONAL SAUDE 01 FORTALEZA,RUA OLAVO BILAC,1200,PRESIDENTE KENNEDY,60320000,-3.717-38.543
SECRETARIA DA SAUDE DO ESTADO DO CEARA,CENTRO ESTADUAL DE REFERENCIA E APOIO A MULHER CERAM,RUAFRANCISCO PINTO,363,BENFICA,60020290,-3.717-38.543
SECRETARIA DA SAUDE DO ESTADO DO CEARA,LACEN LABORATORIO CENTRAL DE SAUDE PUBLICA,AV BARAO DE STUDART,2405,DIONISIO TORRES,60120002,-3.717-38.543
BANCO DE OLHOS DO CEARA B O C EPP,BANCO DE OLHOS DO CEARA,RUA VICENTE NOGUEIRA BRAGA,214,FATIMA,60040570,-3.717-38.543
SMS SECRETARIA MUNICIPAL DE SAUDE FORTALEZA,UAPS MANOEL CARLOS,RUA DESEMBARGADOR FAUSTINO DE ALBUQUERQUE,486,JD DAS OLIVEIRAS,60821440,-3.821946-38.559126
SMS SECRETARIA MUNICIPAL DE SAUDE FORTALEZA,UAPS MACIEL DE BRITO,AV A,S/N,CONJUNTO CEARA,60530590,-3.821946-38.559126


In [18]:
cnes_saude %>% 
    filter(.$latlong_id %in% lats_repetidos_saude$latlong_id & IBGE == '355030' & latlong_id != 'NANA') %>% 
    dplyr::select(c(NO_RAZAO_SOCIAL, NO_FANTASIA, NO_LOGRADOURO, NU_ENDERECO, NO_BAIRRO, CO_CEP, latlong_id)) %>% 
    arrange(latlong_id)

NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_BAIRRO,CO_CEP,latlong_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
PREFEITURA DO MUNICIPIO DE SAO PAULO,UBS J REPUBLICA,AVN GONCALO DE PAIVA GOMES,285,JARDIM REPUBLICA,04812090,-23.3253-46.381
PREFEITURA DO MUNICIPIO DE SAO PAULO,UBS J ARACATI,RUA FRANCISCO HOMEM DE EL REI,87,JARDIM ARACATI,04949070,-23.3253-46.381
PREFEITURA DO MUNICIPIO DE SAO PAULO,SAE DST AIDS CIDADE DUTRA,RUA CRISTINA VASCONCELOS CECCATO,109,CIDADE DUTRA,04802080,-23.3253-46.381
PREFEITURA DO MUNICIPIO DE SAO PAULO,HOSPITAL INTEGRADO SANTO AMARO AMBULATORIO DE ESPECIALIDADE,AV ADOLFO PINHEIRO,339,SANTO AMARO,04753060,-23.3253-46.381
PREFEITURA DO MUNICIPIO DE SAO PAULO,CEO II FREGUESIA DO O BRASILANDIA,RUA RUI DE MORAES APOCALIPSE,02,JARDIM DO TIRO,02842260,-23.473311-46.686246
PREFEITURA DO MUNICIPIO DE SAO PAULO,HOSPITAL DIA DA REDE HORA CERTA BRASILANDIA FO,RUA RUI MORAES DO APOCALIPSE,2,JARDIM DO TIRO,02842260,-23.473311-46.686246
PREFEITURA DO MUNICIPIO DE SAO PAULO,UBS MARIA CECILIA DONNANGELO,RUA RUI DE MORAES APOCALIPSE,02,BRASILANDIA,02842260,-23.473311-46.686246
PREFEITURA DO MUNICIPIO DE SAO PAULO,LABORATORIO MUNICIPAL NOSSA SENHORA DO O,AVENIDA ITABERABA,1377,NOSSA SENHORA DO O,02734000,-23.490081-46.696786
PREFEITURA DO MUNICIPIO DE SAO PAULO,SAE SERVICO DE ASSISTENCIA ESPECIALIZADA NOSSA SENHORA DO O,AVENIDA ITABERABA,1377,FREGUESIA DO O,02734000,-23.490081-46.696786


In [19]:
cnes_saude %>% 
    filter(.$latlong_id %in% lats_repetidos_saude$latlong_id & IBGE == '355030' & latlong_id != 'NANA') %>% 
    dplyr::select(c(NO_RAZAO_SOCIAL, NO_FANTASIA, NO_LOGRADOURO, NU_ENDERECO, NO_BAIRRO, CO_CEP, latlong_id)) %>% 
    arrange(latlong_id) %>% 
    filter(NO_LOGRADOURO == 'RUA RUI DE MORAES APOCALIPSE')

NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_BAIRRO,CO_CEP,latlong_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
PREFEITURA DO MUNICIPIO DE SAO PAULO,CEO II FREGUESIA DO O BRASILANDIA,RUA RUI DE MORAES APOCALIPSE,02,JARDIM DO TIRO,02842260,-23.473311-46.686246
PREFEITURA DO MUNICIPIO DE SAO PAULO,UBS MARIA CECILIA DONNANGELO,RUA RUI DE MORAES APOCALIPSE,02,BRASILANDIA,02842260,-23.473311-46.686246


In [20]:
cnes_saude %>% 
    filter(latlong_id != 'NANA') %>% 
    group_by(latlong_id, IBGE) %>% 
    tally()  %>% 
    arrange(-n) %>% 
    head(20)

latlong_id,IBGE,n
<chr>,<chr>,<int>
-23.548-46.636,355030,471
-22.903-43.208,330455,125
-8.054-34.881,261160,94
-5.795-35.209,240810,76
-19.817-43.956,310620,68
-23.179-45.887,354990,64
-7.115-34.863,250750,61
-5.089-42.802,221100,59
-3.102-60.025,130260,46


In [21]:
cnes_saude %>% filter(latlong_id == '-3.0323795-59.964622') %>% 
    dplyr::select(c(NO_RAZAO_SOCIAL, NO_FANTASIA, NO_LOGRADOURO, NU_ENDERECO, NO_BAIRRO, CO_CEP, latlong_id))

NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_BAIRRO,CO_CEP,latlong_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
MANAUS SECRETARIA MUNICIPAL DE SAUDE,UNIDADE DE SUPORTE BASICO USB 08,RUA 143,S/N,CIDADE NOVA,69099210,-3.0323795-59.964622
MANAUS SECRETARIA MUNICIPAL DE SAUDE,UNIDADE DE SUPORTE BASICO USB 24,RUA 143,S/N,CIDADE NOVA,69099210,-3.0323795-59.964622
MANAUS SECRETARIA MUNICIPAL DE SAUDE,UNIDADE DE SUPORTE BASICO USB 33,RUA 143,33,CIDADE NOVA,69099210,-3.0323795-59.964622
MANAUS SECRETARIA MUNICIPAL DE SAUDE,UNIDADE DE SUPORTE AVANCADO USA 05,RUA 143,S/N,CIDADE NOVA,69099210,-3.0323795-59.964622
MANAUS SECRETARIA MUNICIPAL DE SAUDE,UNIDADE DE SUPORTE BASICO USB 11,RUA 143,S/N,CIDADE NOVA,69099210,-3.0323795-59.964622
MANAUS SECRETARIA MUNICIPAL DE SAUDE,UNIDADE DE SUPORTE BASICO USB 13,AVENIDA BRASIL,3078,COMPENSA,69036115,-3.0323795-59.964622
MANAUS SECRETARIA MUNICIPAL DE SAUDE,UNIDADE DE SUPORTE BASICO USB 05,RUA 143,S/N,CIDADE NOVA,69099210,-3.0323795-59.964622


In [22]:
cnes_saude %>% 
    filter(.$latlong_id %in% lats_repetidos_saude$latlong_id & IBGE == '355030' & latlong_id != 'NANA') %>% 
    dplyr::select(c(NO_RAZAO_SOCIAL, NO_FANTASIA, NO_LOGRADOURO, NU_ENDERECO, NO_BAIRRO, CO_CEP, latlong_id)) %>% 
    group_by(latlong_id, CO_CEP, NO_RAZAO_SOCIAL) %>% 
    tally() %>% 
    filter(n>1)

latlong_id,CO_CEP,NO_RAZAO_SOCIAL,n
<chr>,<chr>,<chr>,<int>
-23.473311-46.686246,02842260,PREFEITURA DO MUNICIPIO DE SAO PAULO,3
-23.490081-46.696786,02734000,PREFEITURA DO MUNICIPIO DE SAO PAULO,2
-23.490468-46.591484,02203010,PREFEITURA DO MUNICIPIO DE SAO PAULO,2
-23.494312-46.400446,08110460,PREFEITURA DO MUNICIPIO DE SAO PAULO,2
-23.494939-46.474189,03807380,PREFEITURA DO MUNICIPIO DE SAO PAULO,2
-23.496938-46.44066,08010150,PREFEITURA DO MUNICIPIO DE SAO PAULO,2
-23.506287-46.697449,02910015,PREFEITURA DO MUNICIPIO DE SAO PAULO,2
-23.524227-46.700973,05049000,PREFEITURA DO MUNICIPIO DE SAO PAULO,2
-23.548-46.636,01008000,PREFEITURA DO MUNICIPIO DE SAO PAULO,2


# 08_cras_assist_social

In [11]:
open_file <- '../../../indice-mobilidade_dados/08_cras_assist_social/2019/cras_2019_geocoded.rds'
cras_assist_social <- read_rds(open_file)

# Criar id único para latlong
cras_assist_social <- 
    cras_assist_social %>% 
    mutate(latitude = str_sub(as.character(lat), start = 0, end = 10),
           longitude = str_sub(as.character(lon), start = 0, end = 10),
           latlong_id = sprintf('%s%s', latitude, longitude))

dim(cras_assist_social)
head(cras_assist_social)

[1] 427  15

code_cras,name_suas,email,telefone,code_muni,endereco,bairro,cep,name_muni,abrev_estado,lon,lat,latitude,longitude,latlong_id
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
13026002569,CRAS CIDADE NOVA,cras.cidadenova@pmm.am.gov.br,9236363849,1302603,"AVENIDA C, QD 67, 46",CIDADE NOVA,69097748,Manaus,AM,-59.97086,-3.013062,-3.0130617,-59.970860,-3.0130617-59.970860
13026002579,CRAS ALFREDO NASCIMENTO,cras.alfredonascimento@pmm.am.gov.br,9236464857,1302603,"RUA 8, 13",ALFREDO NASCIMENTO,69099435,Manaus,AM,-59.95289,-3.014891,-3.0148912,-59.952886,-3.0148912-59.952886
13026002583,CRAS BETÂNIA,cras.betania@pmm.am.gov.br,9232377678,1302603,"RUA SAO LAZARO, 26",BETÂNIA,69073091,Manaus,AM,-59.99546,-3.132717,-3.1327172,-59.995458,-3.1327172-59.995458
13026002585,CRAS Compensa I,cras.compensa1@pmm.am.gov.br,9236714767,1302603,"RUA DA INDUSTRIA, 20",COMPENSA 1,69030070,Manaus,AM,-60.05063,-3.103053,-3.1030530,-60.050625,-3.1030530-60.050625
13026002589,CRAS CACHOEIRINHA,cras.cachoeirinha@pmm.am.gov.br,9236318607,1302603,"AVENIDA CARVALHO LEAL, 1151",CACHOEIRINHA,69065001,Manaus,AM,-60.00637,-3.125646,-3.1256457,-60.006369,-3.1256457-60.006369
13026002590,CRAS ALVORADA I,cras.alvorada1@pmm.am.gov.br,9232389505,1302603,"RUA ABÍLIO ALENCAR, S/N",ALVORADA I,69043324,Manaus,AM,-60.03667,-3.078295,-3.0782947,-60.036667,-3.0782947-60.036667


In [12]:
cras_assist_social %>% group_by(name_muni) %>% tally()

name_muni,n
<chr>,<int>
Belo Horizonte,34
Campinas,12
Campo Grande,20
Curitiba,39
Fortaleza,27
Goiania,15
Joao Pessoa,12
Londrina,10
Maceio,16


In [13]:
lats_repetidos_cras <- 
    cras_assist_social %>% 
    group_by(latlong_id) %>% 
    tally() %>% 
    filter(n > 1)

dim(lats_repetidos_cras)
head(lats_repetidos_cras, 1)

[1] 0 2

latlong_id,n
<chr>,<int>


In [14]:
cras_assist_social %>% 
    filter(.$latlong_id %in% lats_repetidos_cras$latlong_id) %>% 
    group_by(name_muni) %>% 
    tally()

name_muni,n
<chr>,<int>
